<a href="https://colab.research.google.com/github/aidanmrli/MovieRecommender/blob/main/MovieRecommender_LightGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

In this colab, we will learn about GNN pipelines by walking through the LightGCN model, and using it to recommend new movies to individual users through collaborative filtering.

This MovieLens dataset consists of about 100,000 ratings applied to 9,000 movies by 600 users.

In general, the GNN pipeline works as follows:

1. Generate input graph
2. Graph goes through GNN layers
3. Set of node embeddings is produced as output from GNN 
4. Output of GNN is inputted to "prediction head" function to yield a prediction
5. Predictions are compared against ground-truth labels
6. Loss function & Evaluation

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric
!pip install -q git+https://github.com/snap-stanford/deepsnap.git
!pip install -U -q PyDrive

1.11.0+cu113
     |████████████████████████████████| 7.9 MB 33.3 MB/s 
     |████████████████████████████████| 3.5 MB 38.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 407 kB 20.7 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=f8a3a9e8122fc62d797d959926f3052430e7ceba76a466e886ecb5352607ce9e
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [ ]:
# import required modules
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj

# Loading the Dataset (Preprocessing)

We load the dataset and set ratings >=4 on a 0.5 ~ 5 scale as an edge between users and movies.

We split the edges of the graph **once** using a 80/10/10 train/validation/test split. These sets of edges are disjoint sets.

*   Training set used for optimizing GNN model  parameters
*   Validation set used for tuning hyperparameters/develop model 
*   Once model is finalised, use test set to report final performance of model

This is called a fixed split, but there are other strategies like random splitting where average performance over multiple seeds is measured.

In [ ]:
# download the dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

movie_path = './ml-latest-small/movies.csv'
rating_path = './ml-latest-small/ratings.csv'

Extracting ./ml-latest-small.zip


In [ ]:
# load user and movie nodes
def load_node_csv(path, index_col):
    """Loads csv containing node information

    Args:
        path (str): path to csv file
        index_col (str): column name of index column

    Returns:
        dict: mapping of csv row to node id
    """
    df = pd.read_csv(path, index_col=index_col)
    mapping = {index: i for i, index in enumerate(df.index.unique())}
    return mapping


user_mapping = load_node_csv(rating_path, index_col='userId')
movie_mapping = load_node_csv(movie_path, index_col='movieId')

In the below block, we manipulate the graph structure by only adding edges with rating >= 4 out of 5 (originally on a 0.5–5 scale). 

*   Helps facilitate efficient message passing in the computation graph (not too many edges) 
*   Simplifies objective to edge/link prediction between nodes

In [ ]:
# load edges between users and movies
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping, link_index_col, rating_threshold=4):
    """Loads csv containing edges between users and items

    Args:
        path (str): path to csv file
        src_index_col (str): column name of users
        src_mapping (dict): mapping between row number and user id
        dst_index_col (str): column name of items
        dst_mapping (dict): mapping between row number and item id
        link_index_col (str): column name of user item interaction
        rating_threshold (int, optional): Threshold to determine positivity of edge. Defaults to 4.

    Returns:
        torch.Tensor: 2 by N matrix containing the node ids of N user-item edges
    """
    df = pd.read_csv(path)
    edge_index = None
    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_attr = torch.from_numpy(df[link_index_col].values).view(-1, 1).to(torch.long) >= rating_threshold


    edge_index = [[], []]
    for i in range(edge_attr.shape[0]):
        if edge_attr[i]:
            edge_index[0].append(src[i])
            edge_index[1].append(dst[i])

    return torch.tensor(edge_index)


edge_index = load_edge_csv(
    rating_path,
    src_index_col='userId',
    src_mapping=user_mapping,
    dst_index_col='movieId',
    dst_mapping=movie_mapping,
    link_index_col='rating',
    rating_threshold=4,
)

### Notes about graph structure:

We have a **homogeneous graph** (no distinction between the user and movie nodes in the graph structure). 

Graph is **bipartite**, with edges existing only between users and movies.

The adjacency matrix of our homogeneous graph representation is sparse (most elements are 0) so we can represent our adjacency matrix with a PyG `SparseTensor` to reduce memory overheads.

The MessagePassing interface of PyG relies on a gather-scatter scheme to aggregate messages from neighboring nodes. However, it has the disadvantage of explicitly materializing source & target node features, resulting in a high memory footprint on large and dense graphs.

In some cases, like LightGCN, we can represent a GNN by a sparse-matrix multiplication, resulting in a lower memory footprint and a faster execution time. 

Sparse-matrix multiplication is suitable for GNNs if, in message computation, they:
1. Do not use central node features `x_i`
2. Do not use multi-dimensional edge features



### Splitting the graph

Since we are working on an edge prediction task, we choose to transductively split our input graph by edges.

Transductive splitting means the entire input graph is used in all of the training, validation, and test splits. We use the entire input graph to compute the embeddings in training, and only train using the labels of edges in the training set. Likewise, only the labels of validation edges are used to evaluate the validation set.

This has the benefit of retaining all the information in the graph. However, there may be some data leakage between the sets.
<!-- 
In theory, some of the training edges should be assigned as "training supervision edges", and hidden from the GNN in training. So the hierarchy of edges is as follows:

1.   Training: Use training message edges to predict supervision edges
2.   Validation: Use training message & supervision edges to predict validation edges
3. Test: Use all non-test edges to predict test edges
 -->


In [ ]:
# split the edges of the graph using a 80/10/10 train/validation/test split
num_users, num_movies = len(user_mapping), len(movie_mapping)
num_interactions = edge_index.shape[1]
all_indices = [i for i in range(num_interactions)]

train_indices, test_indices = train_test_split(
    all_indices, test_size=0.2, random_state=1)
val_indices, test_indices = train_test_split(
    test_indices, test_size=0.5, random_state=1)

train_edge_index = edge_index[:, train_indices]
val_edge_index = edge_index[:, val_indices]
test_edge_index = edge_index[:, test_indices]

In [ ]:
# convert edge indices into Sparse Tensors: https://pytorch-geometric.readthedocs.io/en/latest/notes/sparse_tensor.html
train_sparse_edge_index = SparseTensor(row=train_edge_index[0], col=train_edge_index[1], sparse_sizes=(
    num_users + num_movies, num_users + num_movies))
val_sparse_edge_index = SparseTensor(row=val_edge_index[0], col=val_edge_index[1], sparse_sizes=(
    num_users + num_movies, num_users + num_movies))
test_sparse_edge_index = SparseTensor(row=test_edge_index[0], col=test_edge_index[1], sparse_sizes=(
    num_users + num_movies, num_users + num_movies))

To leverage sparse-matrix multiplications, the MessagePassing interface introduces the message_and_aggregate() function (which fuses the message() and aggregate() functions into a single computation step), which gets called whenever it is implemented and receives a SparseTensor as input for edge_index.

# Implementing LightGCN

## Idea behind Light Graph Convolution Networks

The message and aggregation functions of a specific GNN propagate information at each layer. 
LightGCN simplifies propagation by *removing* the non-linearity function, feature transformation matrices, and skip connection(s). 

The LightGCN authors argue feature transformation and nonlinear activation (designs common in GCN) contribute little to the performance of collaborative filtering. In fact, including them adds to the difficulty of training and may degrade recommendation performance. LightGCN includes only the most essential component in GCN -- neighborhood aggregation.

For this use case, since the inputs to collaborative filtering are merely the node ids (without rich features), performing multiple non-linear transformations will not contribute to better performance.


## Layer Combination
The only trainable parameters of LightGCN are the 0-th layer embeddings $e_u^{(0)}$ and $e_i^{(0)}$ for each user and item. 

After k-levels of simple diffusion propagation, the final embedding of nodes will be a weighted average of the embeddings at every layer. This helps to approximate self-loops in nodes where the final embedding is informed by the embedding of the node itself and the neighbors and also helps to prevent over-smoothing.

We combine the embeddings obtained at each layer of propagation to form the final embeddings for all user and item, $e_u$ and $e_i$ via the following equation.

\begin{equation}
e_u = \sum_{k = 0}^K \alpha_k e_u^{(k)} \quad e_i = \sum_{k = 0}^K \alpha_k e_i^{(k)}
\end{equation}

$\alpha_k$ : hyperparameter which weights the contribution of the k-th layer embedding to the final embedding

## Model Prediction
The model prediction (Prediction Head Function) is obtained by taking the inner product of the final embeddings of users and movies. 

\begin{equation}
\hat{y}_{ui} = e_u^Te_i
\end{equation}

## Matrix Form
This implementation uses the matrix form of LightGCN. We perform multi-scale diffusion to obtain the final embedding, which sums embeddings diffused across multi-hop scales. 

In [ ]:
# defines LightGCN model
class LightGCN(MessagePassing):
    """LightGCN Model as proposed in https://arxiv.org/abs/2002.02126
    """

    def __init__(self, num_users, num_items, embedding_dim=64, K=3, add_self_loops=False):
        """Initializes LightGCN Model

        Args:
            num_users (int): Number of users
            num_items (int): Number of items
            embedding_dim (int, optional): Dimensionality of embeddings. Defaults to 8.
            K (int, optional): Number of message passing layers. Defaults to 3.
            add_self_loops (bool, optional): Whether to add self loops for message passing. Defaults to False.
        """
        super().__init__()
        self.num_users, self.num_items = num_users, num_items
        self.embedding_dim, self.K = embedding_dim, K
        self.add_self_loops = add_self_loops

        self.users_emb = nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.embedding_dim) # e_u^0
        self.items_emb = nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.embedding_dim) # e_i^0

        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)

    def forward(self, edge_index: SparseTensor):
        """Forward propagation of LightGCN Model.

        Args:
            edge_index (SparseTensor): adjacency matrix

        Returns:
            tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0
        """
        # compute \tilde{A}: symmetrically normalized adjacency matrix
        edge_index_norm = gcn_norm(
            edge_index, add_self_loops=self.add_self_loops)

        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0
        embs = [emb_0]
        emb_k = emb_0

        # multi-scale diffusion
        for i in range(self.K):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)

        embs = torch.stack(embs, dim=1)
        emb_final = torch.mean(embs, dim=1) # E^K

        users_emb_final, items_emb_final = torch.split(
            emb_final, [self.num_users, self.num_items]) # splits into e_u^K and e_i^K

        # returns e_u^K, e_u^0, e_i^K, e_i^0
        return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight

    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        # computes \tilde{A} @ x
        return matmul(adj_t, x)

model = LightGCN(num_users, num_movies)

# Loss Function

A loss function is used to measure the discrepancy between the model's predictions and the ground-truth labels. From there, we backpropagate to the parameters of the neural network, fine-tuning it to optimise the loss.

We utilize a Bayesian Personalized Ranking (BPR) loss, a pairwise objective which encourages the predictions of positive samples to be higher than negative samples for each user.

We use PyG’s `structured_negative_sampling` function to conduct negative sampling which we use to compute BPR loss along with the positive samples.

In [ ]:
# function which random samples a mini-batch of positive and negative samples
def sample_mini_batch(batch_size, edge_index):
    """Randomly samples indices of a minibatch given an adjacency matrix

    Args:
        batch_size (int): minibatch size
        edge_index (torch.Tensor): 2 by N list of edges

    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

In [ ]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """Bayesian Personalized Ranking Loss as described in https://arxiv.org/abs/1205.2618

    Args:
        users_emb_final (torch.Tensor): e_u_k
        users_emb_0 (torch.Tensor): e_u_0
        pos_items_emb_final (torch.Tensor): positive e_i_k
        pos_items_emb_0 (torch.Tensor): positive e_i_0
        neg_items_emb_final (torch.Tensor): negative e_i_k
        neg_items_emb_0 (torch.Tensor): negative e_i_0
        lambda_val (float): lambda value for regularization loss term

    Returns:
        torch.Tensor: scalar bpr loss value
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2)) # L2 loss

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1) # predicted scores of positive samples
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1) # predicted scores of negative samples

    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

In [ ]:
# helper function to get N_u
def get_user_positive_items(edge_index):
    """Generates dictionary of positive items for each user

    Args:
        edge_index (torch.Tensor): 2 by N list of edges

    Returns:
        dict: dictionary of positive items for each user
    """
    user_pos_items = {}
    for i in range(edge_index.shape[1]):
        user = edge_index[0][i].item()
        item = edge_index[1][i].item()
        if user not in user_pos_items:
            user_pos_items[user] = []
        user_pos_items[user].append(item)
    return user_pos_items

# Evaluation

We will use a few metrics to evaluate the model:


*   Recall: $\frac{TP}{TP + FN}$

*   Precision: $\frac{TP}{TP + FP}$

* Normalized Discounted Cumulative Gain (NDCG): takes into account the ordered ranking of recommendations. Recall and Precision are order invariant.


In [ ]:
# computes recall@K and precision@K
def RecallPrecision_ATk(groundTruth, r, k):
    """Computers recall @ k and precision @ k

    Args:
        groundTruth (list): list of lists containing highly rated items of each user
        r (list): list of lists indicating whether each top k item recommended to each user
            is a top k ground truth item or not
        k (intg): determines the top k items to compute precision and recall on

    Returns:
        tuple: recall @ k, precision @ k
    """
    num_correct_pred = torch.sum(r, dim=-1)  # number of correctly predicted items per user
    # number of items liked by each user in the test set
    user_num_liked = torch.Tensor([len(groundTruth[i])
                                  for i in range(len(groundTruth))])
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    return recall.item(), precision.item()

In [ ]:
# computes NDCG@K
def NDCGatK_r(groundTruth, r, k):
    """Computes Normalized Discounted Cumulative Gain (NDCG) @ k

    Args:
        groundTruth (list): list of lists containing highly rated items of each user
        r (list): list of lists indicating whether each top k item recommended to each user
            is a top k ground truth item or not
        k (int): determines the top k items to compute ndcg on

    Returns:
        float: ndcg @ k
    """
    assert len(r) == len(groundTruth)

    test_matrix = torch.zeros((len(r), k))

    for i, items in enumerate(groundTruth):
        length = min(len(items), k)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = torch.sum(max_r * 1. / torch.log2(torch.arange(2, k + 2)), axis=1)
    dcg = r * (1. / torch.log2(torch.arange(2, k + 2)))
    dcg = torch.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg / idcg
    ndcg[torch.isnan(ndcg)] = 0.
    return torch.mean(ndcg).item()

In [ ]:
# wrapper function to get evaluation metrics
def get_metrics(model, edge_index, exclude_edge_indices, k):
    """Computes the evaluation metrics: recall, precision, and ndcg @ k

    Args:
        model (LighGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on

    Returns:
        tuple: recall @ k, precision @ k, ndcg @ k
    """
    user_embedding = model.users_emb.weight
    item_embedding = model.items_emb.weight

    # get ratings between every user and item - shape is num users x num movies
    rating = torch.matmul(user_embedding, item_embedding.T)

    for exclude_edge_index in exclude_edge_indices:
        # gets all the positive items for each user from the edge index
        user_pos_items = get_user_positive_items(exclude_edge_index)
        # get coordinates of all edges to exclude
        exclude_users = []
        exclude_items = []
        for user, items in user_pos_items.items():
            exclude_users.extend([user] * len(items))
            exclude_items.extend(items)

        # set ratings of excluded edges to large negative value
        rating[exclude_users, exclude_items] = -(1 << 10)

    # get the top k recommended items for each user
    _, top_K_items = torch.topk(rating, k=k)

    # get all unique users in evaluated split
    users = edge_index[0].unique()

    test_user_pos_items = get_user_positive_items(edge_index)

    # convert test user pos items dictionary into a list
    test_user_pos_items_list = [
        test_user_pos_items[user.item()] for user in users]

    # determine the correctness of topk predictions
    r = []
    for user in users:
        ground_truth_items = test_user_pos_items[user.item()]
        label = list(map(lambda x: x in ground_truth_items, top_K_items[user]))
        r.append(label)
    r = torch.Tensor(np.array(r).astype('float'))

    recall, precision = RecallPrecision_ATk(test_user_pos_items_list, r, k)
    ndcg = NDCGatK_r(test_user_pos_items_list, r, k)

    return recall, precision, ndcg

In [ ]:
# wrapper function to evaluate model
def evaluation(model, edge_index, sparse_edge_index, exclude_edge_indices, k, lambda_val):
    """Evaluates model loss and metrics including recall, precision, ndcg @ k

    Args:
        model (LightGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        sparse_edge_index (sparseTensor): sparse adjacency matrix for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on
        lambda_val (float): determines lambda for bpr loss

    Returns:
        tuple: bpr loss, recall @ k, precision @ k, ndcg @ k
    """
    # get embeddings
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
        sparse_edge_index)
    edges = structured_negative_sampling(
        edge_index, contains_neg_self_loops=False)
    user_indices, pos_item_indices, neg_item_indices = edges[0], edges[1], edges[2]
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[
        pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[
        neg_item_indices], items_emb_0[neg_item_indices]

    loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0,
                    neg_items_emb_final, neg_items_emb_0, lambda_val).item()

    recall, precision, ndcg = get_metrics(
        model, edge_index, exclude_edge_indices, k)

    return loss, recall, precision, ndcg

# Training

Your test set performance should be in line with the following (*K=20*):

*Recall@K: 0.13, Precision@K: 0.045, NDCG@K: 0.10*

In [ ]:
# define contants
ITERATIONS = 2500
BATCH_SIZE = 1024
LR = 1e-3
ITERS_PER_EVAL = 200
ITERS_PER_LR_DECAY = 200
K = 20
LAMBDA = 1e-6

In [ ]:
# setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}.")


model = model.to(device)
model.train()

optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

edge_index = edge_index.to(device)
train_edge_index = train_edge_index.to(device)
train_sparse_edge_index = train_sparse_edge_index.to(device)

val_edge_index = val_edge_index.to(device)
val_sparse_edge_index = val_sparse_edge_index.to(device)

Using device cuda.


In [ ]:
# training loop
train_losses = []
val_losses = []

for iter in range(ITERATIONS):
    # forward propagation
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
        train_sparse_edge_index)

    # mini batching
    user_indices, pos_item_indices, neg_item_indices = sample_mini_batch(
        BATCH_SIZE, train_edge_index)
    user_indices, pos_item_indices, neg_item_indices = user_indices.to(
        device), pos_item_indices.to(device), neg_item_indices.to(device)
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[
        pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[
        neg_item_indices], items_emb_0[neg_item_indices]

    # loss computation
    train_loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final,
                          pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, LAMBDA)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if iter % ITERS_PER_EVAL == 0:
        model.eval()
        val_loss, recall, precision, ndcg = evaluation(
            model, val_edge_index, val_sparse_edge_index, [train_edge_index], K, LAMBDA)
        print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss, 5)}, val_recall@{K}: {round(recall, 5)}, val_precision@{K}: {round(precision, 5)}, val_ndcg@{K}: {round(ndcg, 5)}")
        train_losses.append(train_loss.item())
        val_losses.append(val_loss)
        model.train()

    if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
        scheduler.step()

[Iteration 0/2500] train_loss: -75.71837, val_loss: -63.64639, val_recall@20: 0.15277, val_precision@20: 0.04629, val_ndcg@20: 0.10708
[Iteration 200/2500] train_loss: -82.37093, val_loss: -67.82334, val_recall@20: 0.15449, val_precision@20: 0.04629, val_ndcg@20: 0.10748
[Iteration 400/2500] train_loss: -89.46757, val_loss: -73.83046, val_recall@20: 0.15479, val_precision@20: 0.04647, val_ndcg@20: 0.1075
[Iteration 600/2500] train_loss: -96.74461, val_loss: -78.09077, val_recall@20: 0.15274, val_precision@20: 0.0462, val_ndcg@20: 0.10707


KeyboardInterrupt: ignored

In [ ]:
# evaluate on test set
model.eval()
test_edge_index = test_edge_index.to(device)
test_sparse_edge_index = test_sparse_edge_index.to(device)

test_loss, test_recall, test_precision, test_ndcg = evaluation(
            model, test_edge_index, test_sparse_edge_index, [train_edge_index, val_edge_index], K, LAMBDA)

print(f"[test_loss: {round(test_loss, 5)}, test_recall@{K}: {round(test_recall, 5)}, test_precision@{K}: {round(test_precision, 5)}, test_ndcg@{K}: {round(test_ndcg, 5)}")

[test_loss: -72.16206, test_recall@20: 0.12687, test_precision@20: 0.04683, test_ndcg@20: 0.1014


# Make new recommendations for a given user!

In [ ]:
model.eval()
df = pd.read_csv(movie_path)
movieid_title = pd.Series(df.title.values,index=df.movieId).to_dict()
movieid_genres = pd.Series(df.genres.values,index=df.movieId).to_dict()

user_pos_items = get_user_positive_items(edge_index)

In [ ]:
def make_predictions(user_id, num_recs):
    user = user_mapping[user_id]
    e_u = model.users_emb.weight[user]
    scores = model.items_emb.weight @ e_u

    values, indices = torch.topk(scores, k=len(user_pos_items[user]) + num_recs)

    movies = [index.cpu().item() for index in indices if index in user_pos_items[user]][:num_recs]
    movie_ids = [list(movie_mapping.keys())[list(movie_mapping.values()).index(movie)] for movie in movies]
    titles = [movieid_title[id] for id in movie_ids]
    genres = [movieid_genres[id] for id in movie_ids]

    print(f"Here are some movies that user {user_id} rated highly")
    for i in range(num_recs):
        print(f"title: {titles[i]}, genres: {genres[i]} ")

    print()

    movies = [index.cpu().item() for index in indices if index not in user_pos_items[user]][:num_recs]
    movie_ids = [list(movie_mapping.keys())[list(movie_mapping.values()).index(movie)] for movie in movies]
    titles = [movieid_title[id] for id in movie_ids]
    genres = [movieid_genres[id] for id in movie_ids]

    print(f"Here are some suggested movies for user {user_id}")
    for i in range(num_recs):
        print(f"title: {titles[i]}, genres: {genres[i]} ")

In [ ]:
make_predictions(user_id=420, num_recs=20)

Here are some movies that user 420 rated highly
title: Shawshank Redemption, The (1994), genres: Crime|Drama 
title: Forrest Gump (1994), genres: Comedy|Drama|Romance|War 
title: Pulp Fiction (1994), genres: Comedy|Crime|Drama|Thriller 
title: Matrix, The (1999), genres: Action|Sci-Fi|Thriller 
title: Fight Club (1999), genres: Action|Crime|Drama|Thriller 
title: Schindler's List (1993), genres: Drama|War 
title: Toy Story (1995), genres: Adventure|Animation|Children|Comedy|Fantasy 
title: Lord of the Rings: The Fellowship of the Ring, The (2001), genres: Adventure|Fantasy 
title: American Beauty (1999), genres: Drama|Romance 
title: Lord of the Rings: The Return of the King, The (2003), genres: Action|Adventure|Drama|Fantasy 
title: Lord of the Rings: The Two Towers, The (2002), genres: Adventure|Fantasy 
title: Aladdin (1992), genres: Adventure|Animation|Children|Comedy|Musical 
title: Sixth Sense, The (1999), genres: Drama|Horror|Mystery 
title: Lion King, The (1994), genres: Advent

In [ ]:
make_predictions(user_id=1, num_recs=20)

Here are some movies that user 1 rated highly
title: Forrest Gump (1994), genres: Comedy|Drama|Romance|War 
title: Silence of the Lambs, The (1991), genres: Crime|Horror|Thriller 
title: Matrix, The (1999), genres: Action|Sci-Fi|Thriller 
title: Star Wars: Episode IV - A New Hope (1977), genres: Action|Adventure|Sci-Fi 
title: Fight Club (1999), genres: Action|Crime|Drama|Thriller 
title: Usual Suspects, The (1995), genres: Crime|Mystery|Thriller 
title: Schindler's List (1993), genres: Drama|War 
title: Star Wars: Episode V - The Empire Strikes Back (1980), genres: Action|Adventure|Sci-Fi 
title: Braveheart (1995), genres: Action|Drama|War 
title: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), genres: Action|Adventure 
title: Star Wars: Episode VI - Return of the Jedi (1983), genres: Action|Adventure|Sci-Fi 
title: Toy Story (1995), genres: Adventure|Animation|Children|Comedy|Fantasy 
title: Fugitive, The (1993), genres: Thriller 
title: Jurassic Park 

In [ ]:
make_predictions(user_id=95, num_recs=20)

Here are some movies that user 95 rated highly
title: Matrix, The (1999), genres: Action|Sci-Fi|Thriller 
title: Star Wars: Episode IV - A New Hope (1977), genres: Action|Adventure|Sci-Fi 
title: Star Wars: Episode V - The Empire Strikes Back (1980), genres: Action|Adventure|Sci-Fi 
title: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), genres: Action|Adventure 
title: Star Wars: Episode VI - Return of the Jedi (1983), genres: Action|Adventure|Sci-Fi 
title: Terminator 2: Judgment Day (1991), genres: Action|Sci-Fi 
title: Saving Private Ryan (1998), genres: Action|Drama|War 
title: Fargo (1996), genres: Comedy|Crime|Drama|Thriller 
title: Lord of the Rings: The Two Towers, The (2002), genres: Adventure|Fantasy 
title: Back to the Future (1985), genres: Adventure|Comedy|Sci-Fi 
title: Eternal Sunshine of the Spotless Mind (2004), genres: Drama|Romance|Sci-Fi 
title: Godfather: Part II, The (1974), genres: Crime|Drama 
title: Twelve Monkeys (a.k.a. 12 Monk

In [ ]:
make_predictions(user_id=580, num_recs=20)

Here are some movies that user 580 rated highly
title: Shawshank Redemption, The (1994), genres: Crime|Drama 
title: Forrest Gump (1994), genres: Comedy|Drama|Romance|War 
title: Pulp Fiction (1994), genres: Comedy|Crime|Drama|Thriller 
title: Matrix, The (1999), genres: Action|Sci-Fi|Thriller 
title: Silence of the Lambs, The (1991), genres: Crime|Horror|Thriller 
title: Star Wars: Episode IV - A New Hope (1977), genres: Action|Adventure|Sci-Fi 
title: Fight Club (1999), genres: Action|Crime|Drama|Thriller 
title: Braveheart (1995), genres: Action|Drama|War 
title: Godfather, The (1972), genres: Crime|Drama 
title: Star Wars: Episode V - The Empire Strikes Back (1980), genres: Action|Adventure|Sci-Fi 
title: Usual Suspects, The (1995), genres: Crime|Mystery|Thriller 
title: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), genres: Action|Adventure 
title: American Beauty (1999), genres: Drama|Romance 
title: Star Wars: Episode VI - Return of the Jedi (198

Out of curiosity, let's see if LightGCN did a better job than simply recommending all users the top 20 rated movies of all time. We have about 600 users, so I will define the top 20 movies as the ones with the highest average rating that have at least 30 ratings (5% of our user population).

In [ ]:
# get df of the top 20 rated movies on average
rating_df = pd.read_csv(rating_path)
size_df = rating_df.groupby('movieId').size().to_frame('size')
rating_df = rating_df.groupby('movieId').agg('mean').join(other=size_df)
rating_df = rating_df[rating_df['size'] >= 30].sort_values(by='rating', ascending=False)
rating_df['movieId'] = rating_df.index
rating_df = rating_df.head(20)
top_movie_id_list = rating_df['movieId'].tolist()

In [ ]:
# get all unique users in evaluated split
users = edge_index[0].unique()

test_user_pos_items = get_user_positive_items(edge_index)

# convert test user pos items dictionary into a list
test_user_pos_items_list = [
    test_user_pos_items[user.item()] for user in users]

# determine the correctness of top20 predictions
r = []
for user in users:
    ground_truth_items = test_user_pos_items[user.item()]
    label = list(map(lambda x: x in ground_truth_items, top_movie_id_list))
    r.append(label)
r = torch.Tensor(np.array(r).astype('float'))

recall, precision = RecallPrecision_ATk(test_user_pos_items_list, r, 20)
ndcg = NDCGatK_r(test_user_pos_items_list, r, 20)

print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'NDCG: {ndcg}')

Recall: 0.004841301124542952
Precision: 0.017323480919003487
NDCG: 0.01571757160127163
